In [251]:
#Data collection
#load python packages
import os
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [252]:
os.getcwd()

'C:\\Users\\chris\\Springboard_rep\\Capstone-Project-2'

In [253]:
#loading the dataset of interest
df = pd.read_csv('appstore_games.csv')
#Drop columns URL, Name, Subtitle, Icon URL, Description, Delevoper and Primary Genre
df.drop(['URL', 'Name', 'Subtitle', 'Icon URL', 'Description', 'Developer', 'Primary Genre'], axis=1, inplace=True)
df.rename(columns = {'Average User Rating':'Rating', 'User Rating Count': 'Rating Count', 'In-app Purchases': 'InAppPurc',
                     'Original Release Date': 'Org Rel Date', 'Current Version Release Date':'Cur Rel Date',
                     'Languages':'Lang'}, inplace=True)

In [254]:
#check for duplicates on ID
#acting strange
df_sort = df.sort_values(by='ID')
duplicateRowsDF = df_sort[df_sort.duplicated()]
duplicateRowsDF

,ID,Rating,Rating Count,Price,InAppPurc,Age Rating,Lang,Size,Genres,Org Rel Date,Cur Rel Date
15,289217958,3.0,112.0,0.00,0.99,4+,"EN, FR, DE, IT, ES",122826752.0,"Games, Strategy, Board",31/08/2008,6/04/2015
57,302532668,NaN,NaN,3.99,NaN,4+,"EN, JA",29797376.0,"Games, Strategy, Board",14/07/2011,13/10/2018
124,321026028,4.5,36581.0,0.00,NaN,4+,EN,69393408.0,"Games, Entertainment, Board, Strategy",4/07/2009,14/06/2019
235,348741265,3.0,1229.0,0.00,"37.99, 199.99, 0.99",12+,"ZH, EN, FR, DE, IT, JA, ES",10781853.0,"Medical, Adventure, Strategy, Games",13/01/2010,27/02/2010
481,405430582,2.5,243.0,0.00,NaN,4+,EN,45953024.0,"Games, Entertainment, Board, Strategy",4/01/2011,30/01/2019
...,...,...,...,...,...,...,...,...,...,...,...
16471,1459748733,NaN,NaN,0.00,NaN,4+,"EN, JA",42607616.0,"Games, Entertainment, Strategy, Puzzle",15/04/2019,15/04/2019
16483,1460101776,NaN,NaN,1.99,NaN,12+,EN,181692416.0,"Games, Strategy, Entertainment, Action",27/04/2019,27/04/2019
16513,1460730256,NaN,NaN,0.00,NaN,4+,EN,171115520.0,"Games, Adventure, Strategy, Entertainment",26/04/2019,29/06/2019
16561,1462595486,NaN,NaN,0.00,NaN,4+,EN,80356352.0,"Games, Strategy, Puzzle",10/05/2019,20/05/2019


In [255]:
#Filled missing lang fields with "EN" and counted number of languages for each app
df['Lang'].fillna('EN', inplace=True)
df['Lang_count'] = df.Lang.str.count(",") + 1
df.drop(['Lang'],axis=1, inplace=True)

In [256]:
df['InAppPurc'] = df['InAppPurc'].notnull()

In [257]:
#Obtain a unique list of genre categories
genre_list = df['Genres'].tolist()

def convert(g_list, seperator=','):
    return seperator.join(g_list)
genre_list_convert = convert(genre_list)
genre_list_clean = genre_list_convert.split(',')

#Count the number of genres for each app
df['gen_count'] = df.Genres.str.count(",") +1
genre_list = df['Genres'].tolist()

genre_cleaner = sorted(set([g.strip() for g in genre_list_clean]))

print(genre_cleaner, type(genre_cleaner))


['Action', 'Adventure', 'Animals & Nature', 'Art', 'Board', 'Books', 'Business', 'Card', 'Casino', 'Casual', 'Comics & Cartoons', 'Education', 'Emoji & Expressions', 'Entertainment', 'Family', 'Finance', 'Food & Drink', 'Games', 'Gaming', 'Health & Fitness', 'Kids & Family', 'Lifestyle', 'Magazines & Newspapers', 'Medical', 'Music', 'Navigation', 'News', 'People', 'Photo & Video', 'Places & Objects', 'Productivity', 'Puzzle', 'Racing', 'Reference', 'Role Playing', 'Shopping', 'Simulation', 'Social Networking', 'Sports', 'Sports & Activities', 'Stickers', 'Strategy', 'Travel', 'Trivia', 'Utilities', 'Weather', 'Word'] <class 'list'>


In [258]:
#Inspect df
print(df)
print(df.info())
print(df.describe().T)

               ID  Rating  Rating Count  Price  InAppPurc Age Rating  \
0       284921427     4.0        3553.0   2.99      False         4+   
1       284926400     3.5         284.0   1.99      False         4+   
2       284946595     3.0        8376.0   0.00      False         4+   
3       285755462     3.5      190394.0   0.00      False         4+   
4       285831220     3.5          28.0   2.99      False         4+   
...           ...     ...           ...    ...        ...        ...   
17002  1474626442     NaN           NaN   0.00      False         4+   
17003  1474919257     NaN           NaN   0.00      False         4+   
17004  1474962324     NaN           NaN   0.00      False         4+   
17005  1474963671     NaN           NaN   0.00       True         4+   
17006  1475076711     NaN           NaN   0.00      False         4+   

              Size                                  Genres Org Rel Date  \
0       15853568.0                 Games, Strategy, Puzzle  

In [259]:
#what should I do with the apps with no rating and price?

In [260]:
#Filled one missing size fields with mean
df['Size'].fillna((df['Size'].mean()), inplace=True)

In [261]:
#Fill 24 missing Price fields with mean
df['Price'].fillna((df['Price'].mean()), inplace=True)

In [262]:
#Strip off '+' from Age Rating and convert to int
df['Age Rating'] = df['Age Rating'].map(lambda x: x.rstrip('+'))
df['Age Rating'] = df['Age Rating'].astype(str).astype(int)

In [263]:
#Convert Org Rel Date and Cur Rel Date to datetime format

df[['Org Rel Date', 'Cur Rel Date']] = df[['Org Rel Date', 'Cur Rel Date']].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17007 entries, 0 to 17006
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            17007 non-null  int64         
 1   Rating        7561 non-null   float64       
 2   Rating Count  7561 non-null   float64       
 3   Price         17007 non-null  float64       
 4   InAppPurc     17007 non-null  bool          
 5   Age Rating    17007 non-null  int32         
 6   Size          17007 non-null  float64       
 7   Genres        17007 non-null  object        
 8   Org Rel Date  17007 non-null  datetime64[ns]
 9   Cur Rel Date  17007 non-null  datetime64[ns]
 10  Lang_count    17007 non-null  int64         
 11  gen_count     17007 non-null  int64         
dtypes: bool(1), datetime64[ns](2), float64(4), int32(1), int64(3), object(1)
memory usage: 1.4+ MB


In [264]:
# #Identify the genres of each app and place in df_cat 
df_cat = pd.DataFrame()
df_cat['Action'] = df['Genres'].str.contains('Action')
df_cat['Adventure'] = df['Genres']. str.contains('Adventure')
df_cat['Animals'] = df['Genres'].str.contains('Animals')
df_cat['Art'] = df['Genres'].str.contains('Art')
df_cat['Board'] = df['Genres'].str.contains('Board')
df_cat['Books'] = df['Genres'].str.contains('Books')
df_cat['Business'] = df['Genres'].str.contains('Business')
df_cat['Card'] = df['Genres'].str.contains('Card')
df_cat['Casino'] = df['Genres'].str.contains('Casino')
df_cat['Casual'] = df['Genres'].str.contains('Casual')
df_cat['Comics & Cartons'] = df['Genres'].str.contains('Comics & Cartons')
df_cat['Education'] = df['Genres']. str.contains('Education')
df_cat['Emoji & Expressions'] = df['Genres'].str.contains('Emoji & Expressions')
df_cat['Entertainment'] = df['Genres'].str.contains('Entertainment')
df_cat['Family'] = df['Genres'].str.contains('Family')
df_cat['Finance'] = df['Genres'].str.contains('Finance')
df_cat['Food & Drink'] = df['Genres'].str.contains('Food & Drink')
df_cat['Games'] = df['Genres'].str.contains('Games')
df_cat['Gaming'] = df['Genres'].str.contains('Gaming')
df_cat['Health & Fitness'] = df['Genres'].str.contains('Health & Fitness')
df_cat['Kids & Family'] = df['Genres'].str.contains('Kids & Family')
df_cat['Lifestyle'] = df['Genres']. str.contains('Lifestyle')
df_cat['Magazines & Newspapers'] = df['Genres'].str.contains('Magazines & Newspapers')
df_cat['Medical'] = df['Genres'].str.contains('Medical')
df_cat['Music'] = df['Genres'].str.contains('Music')
df_cat['Navigation'] = df['Genres'].str.contains('Navigation')
df_cat['News'] = df['Genres'].str.contains('News')
df_cat['People'] = df['Genres'].str.contains('People')
df_cat['Photo & Video'] = df['Genres'].str.contains('Photo & Video')
df_cat['Places & Objects'] = df['Genres'].str.contains('Places & Objects')
df_cat['Productivity'] = df['Genres'].str.contains('Productivity')
df_cat['Puzzle'] = df['Genres']. str.contains('Puzzle')
df_cat['Racing'] = df['Genres'].str.contains('Racing')
df_cat['Reference'] = df['Genres'].str.contains('Reference')
df_cat['Role Playing'] = df['Genres'].str.contains('Role Playing')
df_cat['Shopping'] = df['Genres'].str.contains('Shopping')
df_cat['Simulation'] = df['Genres'].str.contains('Simulation')
df_cat['Social Networking'] = df['Genres'].str.contains('Social Networking')
df_cat['Sports'] = df['Genres'].str.contains('Sports')
df_cat['Sports & Activities'] = df['Genres'].str.contains('Sports & Activities')
df_cat['Stickers'] = df['Genres'].str.contains('Stickers')
df_cat['Strategy'] = df['Genres'].str.contains('Strategy')
df_cat['Travel'] = df['Genres'].str.contains('Travel')
df_cat['Trivia'] = df['Genres']. str.contains('Trivia')
df_cat['Utilities'] = df['Genres'].str.contains('Utilities')
df_cat['Weather'] = df['Genres'].str.contains('Weather')
df_cat['Word'] = df['Genres'].str.contains('Word')

In [265]:
#add in ID column to the df_cat 
df_cat = df_cat.join(df['ID'])
first_col = df_cat.pop('ID')
df_cat.insert(0, 'ID', first_col)
#create a genre count for each ID
df_cat['Genre_count'] = df_cat[df_cat ==True].count(axis=1)
#Drop Genre column from df
df.drop(['Genres'],axis=1, inplace=True)
print(df_cat)
print(df_cat.info())

               ID  Action  Adventure  Animals    Art  Board  Books  Business  \
0       284921427   False      False    False  False  False  False     False   
1       284926400   False      False    False  False   True  False     False   
2       284946595   False      False    False  False   True  False     False   
3       285755462   False      False    False  False  False  False     False   
4       285831220   False      False    False  False   True  False     False   
...           ...     ...        ...      ...    ...    ...    ...       ...   
17002  1474626442   False      False    False  False  False  False     False   
17003  1474919257   False      False    False  False  False  False     False   
17004  1474962324   False      False    False  False  False  False     False   
17005  1474963671    True      False    False  False  False  False     False   
17006  1475076711   False      False    False  False  False  False     False   

        Card  Casino  ...  Sports  Spor

In [266]:
#what can I do with this information with genres

In [267]:
print(df.head())
print(df.info())

          ID  Rating  Rating Count  Price  InAppPurc  Age Rating        Size  \
0  284921427     4.0        3553.0   2.99      False           4  15853568.0   
1  284926400     3.5         284.0   1.99      False           4  12328960.0   
2  284946595     3.0        8376.0   0.00      False           4    674816.0   
3  285755462     3.5      190394.0   0.00      False           4  21552128.0   
4  285831220     3.5          28.0   2.99      False           4  34689024.0   

  Org Rel Date Cur Rel Date  Lang_count  gen_count  
0   2008-11-07   2017-05-30          17          3  
1   2008-11-07   2018-05-17           1          3  
2   2008-11-07   2017-05-09           1          3  
3   2008-07-23   2017-05-30          17          3  
4   2008-07-18   2018-07-22          15          4  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17007 entries, 0 to 17006
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----  